In [39]:
team_id = '40'
split = "test_1"

In [40]:
from src.dataloader import base_loader

df, df_test = base_loader(split)

In [41]:
from src.embedding import base_embedder
from sklearn import preprocessing

X = base_embedder(df.words_str.values)
y_reg = df.score_compound.values
y_clf = preprocessing.LabelEncoder().fit_transform(df.sentiment.values)

X_test = base_embedder(df_test.words_str.values)

### Regression

In [42]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_reg, test_size=0.2, random_state=42, stratify=df.sentiment)

# X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

X_resampled = X_train
y_resampled = y_train

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [43]:
from xgboost import XGBRegressor

reg = XGBRegressor().fit(X_train, y_train)

In [44]:
from src.evaluation import eval_reg

eval_reg(reg, datasets)


Split: train
	RMSE: 0.06
	MAE: 0.04

Split: valididation
	RMSE: 0.35
	MAE: 0.27


In [45]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'reg')

### Classification

In [46]:
X_train, X_val, y_train, y_val = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [47]:
from xgboost import XGBClassifier

clf = XGBClassifier().fit(X_train, y_train)

In [48]:
from src.evaluation import eval_clf

eval_clf(clf, datasets)


Split: train
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      4232
           1       0.83      1.00      0.91      4232
           2       0.99      0.95      0.97      4232

    accuracy                           0.93     12696
   macro avg       0.94      0.93      0.93     12696
weighted avg       0.94      0.93      0.93     12696


Split: validation
              precision    recall  f1-score   support

           0       0.66      0.26      0.37        96
           1       0.79      0.91      0.85      1058
           2       0.75      0.57      0.65       446

    accuracy                           0.78      1600
   macro avg       0.73      0.58      0.62      1600
weighted avg       0.77      0.78      0.76      1600



In [49]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'clf')